In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
# os.environ["TAVILY_API_KEY"] = "Tavily key"

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                you are a helpful assistant. 
                I am talking to a stranger online and I want the conversation to be friendly and funny
                suggest different replies to these messages to make it an engaging task
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [7]:
chain = prompt | llm

In [8]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [9]:
config = {"configurable": {"session_id": "abc11"}}

In [11]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hello how are you doing")]},
    config=config,
)

response.content

'Here are some friendly and funny replies to "hello how are you doing":\n\n**Playful:**\n\n* "I\'m doing great, thanks for asking! How about you? Are you ready for some epic online banter?"\n* "Just surviving another day in the digital jungle. How about you? Any exciting adventures planned?"\n* "I\'m doing better than a one-legged man in a kicking contest. How about you?"\n\n**Genuine:**\n\n* "Hey! I\'m doing well, thanks for asking. How are you?"\n* "I\'m good, thanks! Just chilling and catching up on some online shenanigans. What about you?"\n* "I\'m doing alright, just trying to avoid the existential dread of the internet. How are you holding up?"\n\n**Humorous:**\n\n* "I\'m doing fantastic! I just invented a new way to fold laundry that\'s guaranteed to make you jealous.  You\'ll have to wait for the patent though."\n* "I\'m doing great, thanks for asking! I\'m currently training my pet goldfish to do tricks. He\'s not very good at it yet, but he\'s got the spirit."\n* "I\'m doing 

In [12]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    # conversational_rag_chain.invoke(
    # {"input": user_input},
    # config={
    #     "configurable": {"session_id": "abc123"}
    # },  # constructs a key "abc123" in `store`.
    # )["answer"]
    with_message_history.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config,
    )
    for message in store["abc11"].messages:
        if isinstance(message, AIMessage):
            prefix = "AI"
        else:
            prefix = "User"
    
    print(f"{prefix}: {message.content}\n")

AI: Okay, I'm ready for your message!  Give me the message you want to reply to, and I'll come up with some fun and friendly options for you. 😊 


AI: Okay, here are some fun and friendly replies to "hey are you doing":

**Short & Sweet:**

* "Hey! I'm doing great, how about you?"
* "Just chilling, you?"
* "Hey there!  Doing well, thanks for asking."

**Playful:**

* "I'm doing better than a sloth on a treadmill. How about you?"
* "Surviving another day in the digital world.  What about you?"
* "I'm doing great, but I'm starting to think my cat is plotting against me.  He keeps staring at me with that judging look."

**Humorous:**

* "I'm doing fantastic! I just invented a new way to fold laundry that's guaranteed to make you jealous.  You'll have to wait for the patent though."
* "I'm doing great, thanks for asking! I'm currently training my pet goldfish to do tricks. He's not very good at it yet, but he's got the spirit."
* "I'm doing well, but I'm starting to think my cat is plottin

4